In [1]:
from huggingface_hub import login

login()

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

model_name = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 확인할 입력들
inputs_list = ["1", "01", "001", "0001"]

for inp in inputs_list:
    # 토크나이징
    tokens = tokenizer(inp, return_tensors="pt")

    # 토큰 ID를 실제 토큰으로 변환
    tokenized_tokens = tokenizer.convert_ids_to_tokens(tokens["input_ids"].squeeze().tolist())

    # 출력용
    print("입력 문자열:", inp)
    print("토크나이즈 결과 (input_ids):", tokens["input_ids"].tolist())
    print("토크나이즈 결과 (토큰들):", tokenized_tokens)

    # 모델에 입력하여 임베딩 추출
    with torch.no_grad():
        outputs = model(**tokens)

    # 첫 번째 토큰의 히든 상태 벡터 사용
    embedding = outputs.last_hidden_state[:, 0, :].squeeze(0).cpu().numpy()

    # 임베딩 출력
    print("임베딩 벡터 (첫 번째 토큰):", embedding[:5])
    print("-" * 50)


입력 문자열: 1
토크나이즈 결과 (input_ids): [[16]]
토크나이즈 결과 (토큰들): 1
임베딩 벡터 (첫 번째 토큰): [-0.12819165 -0.13020873 -0.31017897  0.01120264  0.03730279]
--------------------------------------------------
입력 문자열: 01
토크나이즈 결과 (input_ids): [[486]]
토크나이즈 결과 (토큰들): 01
임베딩 벡터 (첫 번째 토큰): [-0.13595845 -0.11018885 -0.3201962   0.02278054  0.0911452 ]
--------------------------------------------------
입력 문자열: 001
토크나이즈 결과 (input_ids): [[8298]]
토크나이즈 결과 (토큰들): 001
임베딩 벡터 (첫 번째 토큰): [-0.2284814  -0.06097323 -0.36425766  0.02031684  0.0823623 ]
--------------------------------------------------
입력 문자열: 0001
토크나이즈 결과 (input_ids): [[18005]]
토크나이즈 결과 (토큰들): 0001
임베딩 벡터 (첫 번째 토큰): [-0.13825744 -0.02562953 -0.260816   -0.02646881  0.11830236]
--------------------------------------------------


In [6]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import pandas as pd

# 모델 및 토크나이저 불러오기
model_name = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 저장할 숫자 리스트 생성
numbers = (
    [str(i) for i in range(10)] +      # "0" ~ "9"
    [f"{i:02d}" for i in range(100)] + # "00" ~ "99"
    [f"{i:03d}" for i in range(1000)] + # "000" ~ "999"
    [f"{i:04d}" for i in range(10000)]  # "0000" ~ "9999"
)

# 숫자별 임베딩 저장 리스트
embedding_list = []

# 숫자별 임베딩 추출
for num in numbers:
    # 토큰 변환
    inputs = tokenizer(num, return_tensors="pt")

    # 모델에 입력하여 임베딩 추출
    with torch.no_grad():
        outputs = model(**inputs)

    # 첫 번째 토큰의 히든 상태 벡터 사용
    embedding = outputs.last_hidden_state[:, 0, :].squeeze(0).cpu().numpy()

    # 데이터 저장 (리스트로 변환 후 추가)
    embedding_list.append([num] + embedding.tolist())

# 데이터프레임 생성 및 CSV 저장
embedding_dim = embedding.shape[0]
columns = ["key"] + [f"dim_{i+1}" for i in range(embedding_dim)]
df = pd.DataFrame(embedding_list, columns=columns)
df.to_csv("gpt2_embeddings.csv", index=False)

# 저장된 데이터 확인
print("CSV 파일 저장 완료")


CSV 파일 저장 완료: gpt2_number_embeddings.csv
